In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import glob
import random
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

In [ ]:
vgg16_model = tf.keras.applications.vgg16.VGG16()

In [ ]:
vgg16_model.summary()

In [ ]:
model = Sequential()
for layer in vgg16_model.layers[:-1]:
  model.add(layer)

model.summary()

In [ ]:
for layer in model.layers:
  layer.trainable = False

In [ ]:
model.add(Dense(units=2, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))
if len(physical_devices) > 0:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
# organize data into train, valid and test sets
os.chdir('datasets/dogsVsCats')
if os.path.isdir('train/dog') is False:
  os.makedirs('train/dog')
  os.makedirs('train/cat')
  os.makedirs('valid/dog')
  os.makedirs('valid/cat')
  os.makedirs('test/dog')
  os.makedirs('test/cat')

  for c in random.sample(glob.glob('dataset/cat*'), 500):
    shutil.move(c, 'train/cat')
  for c in random.sample(glob.glob('dataset/dog*'), 500):
    shutil.move(c, 'train/dog')

  for c in random.sample(glob.glob('dataset/cat*'), 100):
    shutil.move(c, 'valid/cat')
  for c in random.sample(glob.glob('dataset/dog*'), 100):
    shutil.move(c, 'valid/dog')

  for c in random.sample(glob.glob('dataset/cat*'), 50):
    shutil.move(c, 'test/cat')
  for c in random.sample(glob.glob('dataset/dog*'), 50):
    shutil.move(c, 'test/dog')
os.chdir('../../')

In [ ]:
train_path = 'datasets/dogsVsCats/train'
valid_path = 'datasets/dogsVsCats/valid'
test_path = 'datasets/dogsVsCats/test'
os.path.isdir(train_path), os.path.isdir(valid_path), os.path.isdir(test_path)

In [ ]:
data_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.vgg16.preprocess_input)

train_batches = data_generator.flow_from_directory(
    directory=train_path, target_size=(224, 224), classes=['cat', 'dog'], batch_size=10)

valid_batches = data_generator.flow_from_directory(
    directory=valid_path, target_size=(224, 224), classes=['cat', 'dog'], batch_size=10)

test_batches = data_generator.flow_from_directory(directory=test_path, target_size=(
    224, 224), classes=['cat', 'dog'], batch_size=10, shuffle=False)

In [ ]:
model.fit(x=train_batches, validation_data=valid_batches, epochs=5, verbose=2)

In [ ]:
predictions = model.predict(x=test_batches, verbose=0)
test_batches.classes

In [ ]:
cm = confusion_matrix(y_true=test_batches.classes,
                      y_pred=np.argmax(predictions, axis=-1))

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
test_batches.class_indices

In [ ]:
plot_confusion_matrix(cm, classes=['cat', 'dog'], title='Confision Matrix')